In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Activation,  MaxPooling2D, BatchNormalization
from keras import regularizers
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
import keras
import numpy as np

# loading the dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalizing the data to help with the training
X_train /= 255
X_test /= 255

# one-hot encoding using keras' numpy-related utilities
n_classes = 10
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)

# import the models of vgg16 for continues training
model = keras.models.load_model("./Models/vgg16.h5")

In [ ]:
model.evaluate(X_test,Y_test)

313/313 [==============================] - 3s 8ms/step - loss: 0.8260 - accuracy: 0.8756


[0.826046884059906, 0.8755999803543091]

In [ ]:
# train with imagedata generator
datagen = ImageDataGenerator(
            rotation_range=25,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=True)  # randomly flip images
datagen.fit(X_train)
batches = 0
for x_batch, y_batch in datagen.flow(X_train, Y_train, batch_size=250):
    adv_x_batch = fast_gradient_method(model,  x_batch, 0.2, norm=2, targeted=False, clip_max=1, clip_min = 0)
    model.fit(np.concatenate((x_batch, adv_x_batch), axis=0),np.concatenate((y_batch, y_batch), axis=0), batch_size= 500, verbose = 0 )
    batches +=1
    
    if batches >200:
        break
model.evaluate(X_test,Y_test)

313/313 [==============================] - 3s 8ms/step - loss: 1.1113 - accuracy: 0.7964


[1.111286997795105, 0.7964000105857849]

In [ ]:
# continue training on trainning set instead of augmented set
x_split = np.split(X_train,200)
y_split = np.split(Y_train,200)
for i in range(len(x_split)):
    x_batch = x_split[i]
    y_batch = y_split[i]
    adv_x_batch = fast_gradient_method(model,  x_batch, 0.2, norm=2, targeted=False, clip_max=1, clip_min = 0)
    model.fit(np.concatenate((x_batch, adv_x_batch), axis=0),np.concatenate((y_batch, y_batch), axis=0), batch_size= 500, verbose = 1 )


1/1 [==============================] - 0s 114ms/step - loss: 0.8558 - accuracy: 0.8620


In [ ]:
model.evaluate(X_test,Y_test)

313/313 [==============================] - 3s 8ms/step - loss: 0.8836 - accuracy: 0.8602


[0.8835809826850891, 0.8601999878883362]

In [ ]:
model.save('robust_vgg16.h5')
